# Connect Intensive - Machine Learning Nanodegree
# Lesson 08: Reinforcement Learning and $k$-Armed Bandit Problems

## Objectives
  - Introduce and define reinforcement learning terminology:
    - timestep $t$
    - actions $A_t$ and rewards $R_t$
    - value $q_*(a)$ and estimated value $Q_t(a)$
    - greedy actions, exploration vs. exploitation
  - Compare Greedy vs. $\epsilon$-Greedy action selection in $k$-Armed Bandit Problems
  
## Prerequisites
  - You should have the following python packages installed:
    - [matplotlib](http://matplotlib.org/index.html)
    - [numpy](http://www.scipy.org/scipylib/download.html)
    - [pandas](http://pandas.pydata.org/getpandas.html)
    - [seaborn](http://seaborn.pydata.org/installing.html)
    
## Acknowledgements
The code from this notebook is adapted from [Shangtong Zhang's python code](https://github.com/ShangtongZhang/reinforcement-learning-an-introduction) that accompanies Chapter 2 of Sutton and Barto's textbook [Reinforcement Learning: an Introduction](https://webdocs.cs.ualberta.ca/~sutton/book/the-book.html). Shangtong Zhang's copyright notice is in the cell below.

In [ ]:
#######################################################################
# Copyright (C) 2016 Shangtong Zhang(zhangshangtong.cpp@gmail.com)    #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################

## Getting Started
As usual, we start by importing some useful libraries and modules. Don't worry if you get a warning message when importing `matplotlib` -- it just needs to build the font cache, and the warning is just to alert you that this may take a while the first time the cell is run.

**Run** the cell below to import useful libraries for this notebook.

In [ ]:
%matplotlib inline
try:
    import numpy as np
    print("Successfully imported numpy! (Version {})".format(np.version.version))
except ImportError:
    print("Could not import numpy!")
    
try:
    import pandas as pd
    print("Successfully imported pandas! (Version {})".format(pd.__version__))
    pd.options.display.max_rows = 10
except ImportError:
    print("Could not import pandas!")

try:
    # Import the seaborn Python visualization library
    import seaborn as sns
    print("Successfully imported seaborn! (Version {})".format(sns.__version__))
except ImportError:
    print("Could not import seaborn!")
    
try:
    import matplotlib
    import matplotlib.pyplot as plt
    plt.style.use('ggplot')
    print("Successfully imported matplotlib.pyplot! (Version {})".format(matplotlib.__version__))
except ImportError:
    print("Could not import matplotlib.pyplot!")
    
try:
    from IPython.display import display
    print("Successfully imported display from IPython.display!")
except ImportError:
    print("Could not import display from IPython.display!")

# The $k$-Armed Bandit Problem

<table>
<tr>
<td><img src="http://www.clker.com/cliparts/5/y/Q/h/g/d/slot-machine-hi.png" alt="Slot Machine" style="background-color:red;width:120px" /></td>
<td><img src="http://www.clker.com/cliparts/5/y/Q/h/g/d/slot-machine-hi.png" alt="Slot Machine" style="background-color:orange;width:120px" /></td>
<td><img src="http://www.clker.com/cliparts/5/y/Q/h/g/d/slot-machine-hi.png" alt="Slot Machine" style="background-color:yellow;width:120px" /></td>
<td><img src="http://www.clker.com/cliparts/5/y/Q/h/g/d/slot-machine-hi.png" alt="Slot Machine" style="background-color:green;width:120px" /></td>
<td><img src="http://www.clker.com/cliparts/5/y/Q/h/g/d/slot-machine-hi.png" alt="Slot Machine" style="background-color:blue;width:120px" /></td>
<td><img src="http://www.clker.com/cliparts/5/y/Q/h/g/d/slot-machine-hi.png" alt="Slot Machine" style="background-color:purple;width:120px" /></td>
</tr>
</table>

Suppose we're at the Reinforcement Learning Casino and we see $k$ different slot machines. During a **time step** $t$, we can select as our **action** $A_t$ one of the $k$ slot machines. We then pull the lever and receive a payout or **reward** $R_t$. Each slot machine pays out winnings based on some **stationary probability distribution**, meaning the likelihood of a certain outcome or reward is **independent** of the current time step. Now we'll keep doing this for many many timesteps (we've got the whole day to spend in the Reinforcement Learning Casino). At each time step, we're faced with the question "Which slot machine should we play to maximize our winnings?" ... how do we answer this question?

If we were omniscient beings, we would know the **value** $q_*(a)$ of each action $a$. The value $q_*(a)$ of action $a$ is defined as the **expected value** of the reward $R_t$, given that action $a$ is selected at time $t$:

$$q_*(a)=\mathbb{E}\left(R_t\vert A_t=a\right).$$

Knowing the value of all actions, we would also know which of the actions is the **optimal action**, or the one which has the greatest value $q_*$. However, we're not omniscient beings, so we'll have to keep track of our observations of actions and rewards. At each time step, we can estimate the value of each action. We'll denote the **estimated value** of action $a$ at time step $t$ as $Q_t(a)$. But how do we compute $Q_t(a)$?

##  Estimating the Value of Actions
After some time at the Reinforcement Learning Casino, we want the estimated values $Q_t(a)$ for each action $a$. One approach is the **sample-average method**:

$$Q_t(a)\ \dot{=}\ \frac{\text{Sum of rewards when }a\text{ is taken prior to }t}{\text{Number of times }a\text{ is taken prior to }t}$$

Basically, we just add up all rewards from selecting action $a$, and divide by the total number of time steps during which action $a$ was selected. It's just the **average** of the **sample** of relevant rewards, hence the name **sample-average method**. A couple points to note:

  - The sample-average method is not the only way to obtain the estimated value $Q_t(a)$, nor is it always the best way.
  - If action $a$ has not yet been taken prior to time step $t$, the denominator in the sample-average method will be zero. In this case, we can define $Q_t(a)$ to be some default value, typically $Q_t(a)=0$.
  
Let's see the sample-average method in action. **Run** the cell below to create a $k$-Armed Bandit Problem with 6 arms (6 actions). Each of these six actions $a$ will have a value $q_*(a)$ drawn from the Normal Distribution $N(0,1)$ with mean 0 and variance 1. We'll choose actions randomly from the set of six arms for the first ten timesteps. The reward $R_t(A_t)$ for each action will be the value $q_*(A_t=a)$, plus a randomly-drawn value from the Normal Distribution $N(0,1)$. The output will be a `DataFrame` object with the timesteps, along with the action and reward for each timestep.

In [ ]:
# Number of arms (actions) for the k-Armed Bandit Problem
k = 6

# How many timesteps or actions to simulate
num_timesteps = 10

# Initialize the random seed for np.random methods
np.random.seed(0)

# Random choice of actions for the first num_timesteps actions
actions = np.random.choice(range(k),num_timesteps,replace=True)

# Initialize the rewards to be an array of zeroes
rewards = np.zeros(num_timesteps)

# Each of the k actions will have a random value drawn from
# the normal distribution with mean zero and variance one.
bandit_means = np.random.randn(k)

# Fill the rewards array based on the bandit_means array,
# but add random values drawn from the normal distribution N(0,1)
for idx, action in enumerate(actions):
    rewards[idx] = bandit_means[action] + np.random.randn()

# Combine the timestep, action, and reward history into a DataFrame
bandit_df = pd.DataFrame({'timestep':np.arange(num_timesteps),
                          'action' :actions,
                          'reward' :rewards},
                         columns=['timestep','action','reward'])
display(bandit_df)

Now **run** the cell below to use the sample-average method to estimate the value $Q_t(a)$ of all six actions after ten time steps.

In [ ]:
display(bandit_df\
        .groupby('action')\
        .mean()\
        .sort_values(by='reward',ascending=False)\
        ['reward'])

It looks like action 0 has the largest estimate, followed by action 4, action 3, action 2, action 5, and lastly, action 1.

## Selecting an Action - Let's be Greedy!

So we've covered how to use the sample-average method to estimate the value $Q_t(a)$ of an action $a$, based on the history of past actions and rewards prior to timestep $t$. At any timestep, there is at least one action $a$ with the largest estimated value $Q_t(a)$. These actions are called **greedy** actions. Now we have a choice: we can either **exploit** our current knowledge and select the greedy action, or we can **explore** by selecting a non-greedy action. Exploring allows us to improve our estimates for the non-greedy actions (and perhaps discover a better action to exploit at later timesteps), while exploitation maximizes the immediate expected reward for the next step. We cannot both explore and exploit during the same timestep; for that reason, people often refer to the **conflict** of exploration vs. exploitation.

The **greedy action selection** method chooses the action $a$ with the largest estimated value $Q_t(a)$ at each time step $t$:

$$A_t\ \dot{=}\ \underset{a}{\arg\max}\ Q_t(a)$$

Looking at the action and reward history for the sample $k$-Armed Bandit Problem generated above, the greedy action selection method suggests we should select action 0. We can see how good of a choice this action really is. **Run** the cell below to display the true values, sorted with the optimal action on top. Did greedy action selection choose the optimal action?

In [ ]:
true_values = pd.Series(data = bandit_means, name = "values", index=pd.Series(np.arange(k),name="action"))
display(true_values.sort_values(ascending=False))

Action 0 is actually ranked second! In this case, greedy action selection misses out on action 5, which has the greatest value $q_*(a)$. In fact, based on the sample-average method, action 5 is currently ranked *fifth* out of six! With only exploitation and no exploration, we will miss out on the optimal action!

Our history of actions and rewards was only ten -- it shouldn't be a surprise that the greedy action selection method couldn't find the optimal action. However, by the [**law of large numbers**](https://en.wikipedia.org/wiki/Law_of_large_numbers), if we go for many many timesteps and take a much larger sample of actions and rewards, the estimated values $Q_t(a)$ should approach the true values $q_*(a)$ for each action $a$. **Run** the cell below to do exactly that: take 10,000 actions and receive 10,000 rewards. A [violin plot](http://seaborn.pydata.org/generated/seaborn.violinplot.html) will depict the reward distributions for each action (Note: you will need to have installed the [seaborn Python visualization library](http://seaborn.pydata.org/index.html) to generate the violin plot). For this much larger history of actions and rewards, do the reward distributions more closely match the true values from the above cell?

In [ ]:
# How many timesteps or actions to simulate
large_num_timesteps = 10000

# Reset the random seed
np.random.seed(0)

# Random choice of actions for the first num_timesteps actions
large_actions = np.random.choice(range(k),large_num_timesteps,replace=True)

# Initialize the rewards to be an array of zeroes
large_rewards = np.zeros(large_num_timesteps)

# Fill the rewards array based on the bandit_means array,
# but add random values drawn from the normal distribution N(0,1)
for idx, action in enumerate(large_actions):
    large_rewards[idx] = bandit_means[action] + np.random.randn()
    
# Combine the timestep, action, and reward history into a DataFrame
large_bandit_df = pd.DataFrame({'timestep':np.arange(large_num_timesteps),
                                 'action' :large_actions,
                                 'reward' :large_rewards},
                                 columns=['timestep','action','reward'])

# Create the violin plot, depicting the reward distribution of each action
ax = sns.violinplot(x="action", y="reward",data=large_bandit_df)

# Set the title of the violin plot
title = ax.set_title("Distribution of Rewards for {}-Armed Bandit Problem".format(k))

## Let's be $\epsilon$-Greedy!

A simple alternative to greedy action selection is the **$\boldsymbol{\epsilon}$-greedy action selection** method. Here, $\epsilon$ is some number between 0 and 1, denoting the probability of choosing an action randomly, *i.e.* taking an *exploration* action. If we don't *explore*, then we *exploit*: the probability that the greedy action will be selected is $1-\epsilon$.

Does $\epsilon$-greedy action selection pay off in the long-term? What about for different values of $\epsilon$? Let's find out! Read through the cell below, coded by Shangtong Zhang (see Acknowledgements), to get an idea of what is contained within the `Bandit` class. The key methods are:
  - `getAction(self)`: determining the next action based on the action-reward history, and whether to explore or exploit.
  - `takeAction(self,action)`: take the prescribed `action`, receive the reward, and update the estimated values $Q_t(a)$ for each action for the next timestep.
  
Once you've read through the code, **run** the cell below to create the `Bandit` class.

In [ ]:
class Bandit:
    # @kArm: Number of arms
    # @epsilon: probability for exploration in epsilon-greedy algorithm (0 <= epsilon <= 1)
    # @initial: initial estimation for each action
    # @stepSize: constant step size for updating estimations
    # @sampleAverages: if True, use sample averages to update estimations instead of constant step size
    # @UCBParam: if not None, use UCB algorithm to select action
    # @gradient: if True, use gradient based bandit algorithm
    # @gradientBaseline: if True, use average reward as baseline for gradient based bandit algorithm
    def __init__(self, kArm=10, epsilon=0, initial=0, stepSize=0.1,\
                 sampleAverages=False, UCBParam=None,\
                 gradient=False, gradientBaseline=False, trueReward=0):
        # Number of arms for the k-Armed Bandit Problem
        self.k = kArm
        
        # Step size used in the gradient based bandit algorithm
        self.stepSize = stepSize
        
        # if True, use sample averages to update estimations instead of constant step size
        self.sampleAverages = sampleAverages
        
        # The indices of all k bandits range from 0 to k-1
        # self.indices is an array with all the bandit indices
        self.indices = np.arange(self.k)
        
        # Initialize the current timestep: time = 0
        self.time = 0
        
        # if not None, use Upper-Confidence-Bound (UCB) algorithm to select action
        self.UCBParam = UCBParam
        
        # if True, use gradient based bandit algorithm
        self.gradient = gradient
        
        # if True, use average reward as baseline for gradient based bandit algorithm
        self.gradientBaseline = gradientBaseline
        self.averageReward = 0
        self.trueReward = trueReward

        # real reward for each action
        self.qTrue = []

        # estimation for each action
        self.qEst = np.zeros(self.k)

        # Number of times each action has been chosen
        self.actionCount = []
        
        # probability for exploration in epsilon-greedy algorithm (0 <= epsilon <= 1)
        self.epsilon = epsilon

        # initialize true expected value of all actions with N(0,1) distribution,
        # initialize the estimations with desired initial value (typically 0),
        # and set all action counts to zero
        for i in range(self.k):
            self.qTrue.append(np.random.randn() + trueReward)
            self.qEst[i] = initial
            self.actionCount.append(0)

        # Determine the best action from the true expected values of all actions
        self.bestAction = np.argmax(self.qTrue)

    # get an action for this bandit, explore or exploit?
    def getAction(self):
        # explore
        if self.epsilon > 0:
            # np.random.binomial(n,p) is the result of flipping a coin n times
            #   probability of heads (1) =  p
            #   probability of tails (0) = 1-p
            if np.random.binomial(1, self.epsilon) == 1:
                # For an exploratory (random) action, shuffle the indices array
                # and return the first index in the shuffled array.
                np.random.shuffle(self.indices)
                return self.indices[0]

        # exploit
        if self.UCBParam is not None:
            # use Upper-Confidence-Bound (UCB) algorithm to select action
            UCBEst = self.qEst + \
                     self.UCBParam * np.sqrt(np.log(self.time + 1) / (np.asarray(self.actionCount) + 1))
            return np.argmax(UCBEst)
        if self.gradient:
            # use gradient based bandit algorithm
            expEst = np.exp(self.qEst)
            self.actionProb = expEst / np.sum(expEst)
            return np.random.choice(self.indices, p=self.actionProb)
        return np.argmax(self.qEst)

    # take an action, update estimation for this action
    def takeAction(self, action):
        # generate the reward under N(real reward, 1)
        reward = np.random.randn() + self.qTrue[action]
        self.time += 1
        self.averageReward = (self.time - 1.0) / self.time * self.averageReward + reward / self.time
        self.actionCount[action] += 1

        if self.sampleAverages:
            # update estimation using sample averages
            self.qEst[action] += 1.0 / self.actionCount[action] * (reward - self.qEst[action])
        elif self.gradient:
            oneHot = np.zeros(self.k)
            oneHot[action] = 1
            if self.gradientBaseline:
                baseline = self.averageReward
            else:
                baseline = 0
            self.qEst = self.qEst + self.stepSize * (reward - baseline) * (oneHot - self.actionProb)
        else:
            # update estimation with constant step size
            self.qEst[action] += 0.1 * (reward - self.qEst[action])
        return reward

Now **run** the cell below to create the `epsilonGreedy` method, which will simulate the $k$-Armed Bandit Problem multiple times for a prescribed number of timesteps and prescribed list of $\epsilon$ values.

In [ ]:
# for figure 2.2 from Sutton and Barto
def epsilonGreedy(nBandits, time, k=10, epsilons = [0.1, 0.01, 0]):
    # @nBandits: (integer) The number of k-Armed Bandit Problems (kABP) to simulate
    # @time: (integer) The number of timesteps to simulate in each kABP
    # @k: (integer) The number of arms (actions) for each kABP
    # @epsilons: (list of floats) Value(s) of epsilon for epsilon greedy selection
    
    # Initialize the figure counter to zero
    figureIndex = 0

    # Initialize a counter for each timestep and for each epsilon to keep track of 
    # the number of times the best action was chosen for each timestep
    bestActionCounts = [np.zeros(time, dtype='float') for _ in range(len(epsilons))]
    
    # Initialize an array for the average reward at each timestep for each epsilon
    averageRewards = [np.zeros(time, dtype='float') for _ in range(len(epsilons))]
    
    # Initialize empty lists, one for each epsilon, that will contain all Bandit objects
    bandits = [list() for _ in range(len(epsilons))]
    
    # Create all nBandits Bandit objects for each value of epsilon,
    # initialize them with k actions, and the correct epsilon value.
    # Set sampleAverages to True - we are calculating qEst from the sample averages
    for i, (eps, bandit) in enumerate(zip(epsilons,bandits)):
        for _ in range(nBandits):
            bandit.append(Bandit(kArm=k, epsilon=eps, sampleAverages=True))
    
    # Now iterate through each Bandit object and take actions for the prescribed
    # number of timesteps. Update the rewards and average rewards, and increment
    # the best action counter if the best action was selected.
    for epsInd in range(len(epsilons)):
        for i in range(nBandits):
            for t in range(time):
                action = bandits[epsInd][i].getAction()
                reward = bandits[epsInd][i].takeAction(action)
                averageRewards[epsInd][t] += reward
                if action == bandits[epsInd][i].bestAction:
                    bestActionCounts[epsInd][t] += 1
    
    # First figure: % of times at each timestep the optimal action was chosen.
    plt.figure(figureIndex)
    figureIndex += 1
    for eps, counts in zip(epsilons, bestActionCounts):
        counts /= (nBandits*0.01)
        plt.plot(counts, label='$\epsilon$ = '+str(eps))
    plt.xlabel('Time Steps')
    plt.ylabel('% Optimal Action')
    plt.ylim([0,100])
    plt.title('% Optimal Action\n({}-Armed Bandit, Average Over {} Runs)'.format(k, nBandits))
    plt.legend(loc='lower right')
    
    # Second figure: average reward at each timestep for each epsilon.
    plt.figure(figureIndex)
    figureIndex += 1
    for eps, rewards in zip(epsilons, averageRewards):
        rewards /= nBandits
        plt.plot(rewards, label='$\epsilon$ = '+str(eps))
    plt.xlabel('Time Steps')
    plt.ylabel('Average Reward')
    plt.title('Average Reward\n({}-Armed Bandit, Average Over {} Runs)'.format(k, nBandits))
    plt.legend(loc='lower right')


Now **run** the cell below to generate a plot of the percentage of times the greedy and $\epsilon$-greedy action selection methods select the optimal action for each timestep. You should see that the greedy action selection method quickly plateaus, so on average it's finding the optimal action fewer than 40% of the time! The $\epsilon$-greedy methods improve over time: with a larger $\epsilon$ of 0.1, the improvement is more rapid, but the upper bound on the percentage of optimal actions for very long times is around 91% in this case. With a smaller $\epsilon$ of 0.01, the improvement at each timestep is slower, but the upper bound on the optimal percentage is closer to 99%. Can you figure out why both of these optimal percentages are bounded?

In [ ]:
np.random.seed(0)
epsilonGreedy(2000,1000,10)

## Under Construction...

In the future, I'd like to add to this notebook to contain a discussion of the rest of the Chapter 2 content from the Sutton and Barto text. For now, I'm including Shangtong Zhang's code as a placeholder.

In [ ]:

# for figure 2.3
def optimisticInitialValues(nBandits, time):
    bandits = [[], []]
    bestActionCounts = [np.zeros(time, dtype='float') for _ in range(0, len(bandits))]
    bandits[0] = [Bandit(epsilon=0, initial=5, stepSize=0.1) for _ in range(0, nBandits)]
    bandits[1] = [Bandit(epsilon=0.1, initial=0, stepSize=0.1) for _ in range(0, nBandits)]
    for banditInd in range(0, len(bandits)):
        for i in range(0, nBandits):
            for t in range(0, time):
                action = bandits[banditInd][i].getAction()
                bandits[banditInd][i].takeAction(action)
                if action == bandits[banditInd][i].bestAction:
                    bestActionCounts[banditInd][t] += 1
    bestActionCounts[0] /= nBandits
    bestActionCounts[1] /= nBandits
    global figureIndex
    plt.figure(figureIndex)
    figureIndex += 1
    plt.plot(bestActionCounts[0], label='epsilon = 0, q = 5')
    plt.plot(bestActionCounts[1], label='epsilon = 0.1, q = 0')
    plt.xlabel('Steps')
    plt.ylabel('% optimal action')
    plt.legend()

# for figure 2.4
def ucb(nBandits, time):
    bandits = [[], []]
    averageRewards = [np.zeros(time, dtype='float') for _ in range(0, len(bandits))]
    bandits[0] = [Bandit(epsilon=0, stepSize=0.1, UCBParam=2) for _ in range(0, nBandits)]
    bandits[1] = [Bandit(epsilon=0.1, stepSize=0.1) for _ in range(0, nBandits)]
    for banditInd in range(0, len(bandits)):
        for i in range(0, nBandits):
            for t in range(0, time):
                action = bandits[banditInd][i].getAction()
                reward = bandits[banditInd][i].takeAction(action)
                averageRewards[banditInd][t] += reward
    averageRewards[0] /= nBandits
    averageRewards[1] /= nBandits
    global figureIndex
    plt.figure(figureIndex)
    figureIndex += 1
    plt.plot(averageRewards[0], label='UCB c = 2')
    plt.plot(averageRewards[1], label='epsilon greedy epsilon = 0.1')
    plt.xlabel('Steps')
    plt.ylabel('Average reward')
    plt.legend()

# for figure 2.5
def gradientBandit(nBandits, time):
    bandits =[[], [], [], []]
    bandits[0] = [Bandit(gradient=True, stepSize=0.1, gradientBaseline=True, trueReward=4) for _ in range(0, nBandits)]
    bandits[1] = [Bandit(gradient=True, stepSize=0.1, gradientBaseline=False, trueReward=4) for _ in range(0, nBandits)]
    bandits[2] = [Bandit(gradient=True, stepSize=0.4, gradientBaseline=True, trueReward=4) for _ in range(0, nBandits)]
    bandits[3] = [Bandit(gradient=True, stepSize=0.4, gradientBaseline=False, trueReward=4) for _ in range(0, nBandits)]
    bestActionCounts = [np.zeros(time, dtype='float') for _ in range(0, len(bandits))]
    for banditInd in range(0, len(bandits)):
        for i in range(0, nBandits):
            for t in range(0, time):
                action = bandits[banditInd][i].getAction()
                bandits[banditInd][i].takeAction(action)
                if action == bandits[banditInd][i].bestAction:
                    bestActionCounts[banditInd][t] += 1
    for counts in bestActionCounts:
        counts /= nBandits
    labels = ['alpha = 0.1, with baseline',
              'alpha = 0.1, without baseline',
              'alpha = 0.4, with baseline',
              'alpha = 0.4, without baseline']
    global figureIndex
    plt.figure(figureIndex)
    figureIndex += 1
    for i in range(0, len(bandits)):
        plt.plot(bestActionCounts[i], label=labels[i])
    plt.xlabel('Steps')
    plt.ylabel('% Optimal action')
    plt.legend()

# Figure 2.6
def figure2_6(nBandits, time):
    labels = ['epsilon-greedy', 'gradient bandit',
              'UCB', 'optimistic initialization']
    generators = [lambda epsilon: Bandit(epsilon=epsilon, sampleAverages=True),
                  lambda alpha: Bandit(gradient=True, stepSize=alpha, gradientBaseline=True),
                  lambda coef: Bandit(epsilon=0, stepSize=0.1, UCBParam=coef),
                  lambda initial: Bandit(epsilon=0, initial=initial, stepSize=0.1)]
    parameters = [np.arange(-7, -1),
                  np.arange(-5, 2),
                  np.arange(-4, 3),
                  np.arange(-2, 3)]
    averageRewards = []
    for i in range(len(labels)):
        averageRewards.append([])
        for param_ in parameters[i]:
            param = pow(2, param_)
            reward = 0.0
            for banditIndex in range(nBandits):
                bandit = generators[i](param)
                print labels[i], 'parameter:', param, str(banditIndex) + 'th bandit'
                for step in range(time):
                    action = bandit.getAction()
                    reward += bandit.takeAction(action)
            reward /= nBandits * time
            averageRewards[-1].append(reward)

    global figureIndex
    plt.figure(figureIndex)
    figureIndex += 1
    for i in range(len(labels)):
        plt.plot(parameters[i], averageRewards[i], label=labels[i])
    plt.xlabel('Parameter(2^x)')
    plt.ylabel('Average reward')
    plt.legend()


# epsilonGreedy(200, 1000)
# optimisticInitialValues(200, 1000)
# ucb(1000, 1000)
# gradientBandit(200, 1000)
# figure2_6(200, 1000)
# plt.show()